In [1]:
import warnings
warnings.filterwarnings(action='ignore')
%config Completer.use_jedi = False
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import pandas as pd

Instructions for updating:
non-resource variables are not supported in the long term


***
지문읽고 주제 분류하기
***

다음과 같은 데이터를 데이터프레임으로 저장한다.  
입력 데이터는 음식 및 스포츠 관련 지문으로 구성되어 있다.  

In [2]:
paragraph_dict_list = [
	{'paragraph': 'Dishplace is located in sunnyvale downtown there is parking around the area but it can be difficult to find during peak business hours my sisters and i came to this place for dinner on a weekday they were really busy so i highly recommended making reservations unless you have the patience to wait', 'category': 'food'},
	{'paragraph': 'Service can be slower during busy hours but our waiter was courteous and help gave some great entree recommendations', 'category': 'food'},
	{'paragraph': 'Portions are huge both french toast and their various omelettes are really good their french toast is probably 1.5x more than other brunch places great place to visit if you are hungry and dont want to wait 1 hour for a table', 'category': 'food'},
	{'paragraph': 'We started with apps going the chicken and waffle slides and chicken nachos the sliders were amazing and the nachos were good too maybe by themselves the nachos would have scored better but after those sliders they were up against some tough competition', 'category': 'food'},
	{'paragraph': 'The biscuits and gravy was too salty two people in my group had the gravy and all thought it was too salty my hubby ordered a side of double egg and it was served on two small plates who serves eggs to one person on separate plates we commented on that when it was delivered and even the server laughed and said she doesnt know why the kitchen does that presentation of food is important and they really missed on this one', 'category': 'food'},
	{'paragraph': 'The garlic fries were a great starter (and a happy hour special) the pancakes looked and tasted great and were a fairly generous portion', 'category': 'food'},
	{'paragraph': 'Our meal was excellent i had the pasta ai formaggi which was so rich i didnt dare eat it all although i certainly wanted to excellent flavors with a great texture contrast between the soft pasta and the crisp bread crumbs too much sauce for me but a wonderful dish', 'category': 'food'},
	{'paragraph': 'What i enjoy most about palo alto is so many restaurants have dog-friendly seating outside i had bookmarked italico from when they first opened about a 1.5 years ago and was jonesing for some pasta so time to finally knock that bookmark off', 'category': 'food'},
	{'paragraph': 'The drinks came out fairly quickly a good two to three minutes after the orders were taken i expected my iced tea to taste a bit more sweet but this was straight up green tea with ice in it not to complain of course but i was pleasantly surprised', 'category': 'food'},
	{'paragraph': 'Despite the not so good burger the service was so slow the restaurant wasnt even half full and they took very long from the moment we got seated to the time we left it was almost 2 hours we thought that it would be quick since we ordered as soon as we sat down my coworkers did seem to enjoy their beef burgers for those who eat beef however i will not be returning it is too expensive and extremely slow service', 'category': 'food'},
	{'paragraph': 'The four reigning major champions simona halep caroline wozniacki angelique kerber and defending us open champion sloane stephens could make a case for being the quartet most likely to succeed especially as all but stephens has also enjoyed the no1 ranking within the last 14 months as they prepare for their gruelling new york campaigns they currently hold the top four places in the ranks', 'category': 'sports'},
	{'paragraph': 'The briton was seeded nn7 here last year before a slump in form and confidence took her down to no46 after five first-round losses but there have been signs of a turnaround including a victory over a sub-par serena williams in san jose plus wins against jelena ostapenko and victoria azarenka in montreal. konta pulled out of new haven this week with illness but will hope for good things where she first scored wins in a major before her big breakthroughs to the semis in australia and wimbledon', 'category': 'sports'},
	{'paragraph': 'Stephens surged her way back from injury in stunning style to win her first major here last year—and ranked just no83 she has since proved what a big time player she is winning the miami title via four fellow major champions then reaching the final at the french open back on north american hard courts she ran to the final in montreal only just edged out by halep she has also avoided many of the big names in her quarter—except for wild card azarenka as a possible in the third round', 'category': 'sports'},
	{'paragraph': 'When it came to england chances in the world cup it would be fair to say that most fans had never been more pessimistic than they were this year after enduring years of truly dismal performances at major tournaments – culminating in the 2014 event where they failed to win any of their three group games and finished in bottom spot those results led to the resignation of manager roy hodgson', 'category': 'sports'},
	{'paragraph': 'The team that eliminated russia – croatia – also improved enormously during the tournament before it began their odds were 33/1 but they played with real flair and star players like luka modric ivan rakitic and ivan perisic showed their quality on the world stage having displayed their potential by winning all three of their group stage games croatia went on to face difficult tests like the semi-final against england', 'category': 'sports'},
	{'paragraph': 'The perseyside outfit finished in fourth place in the premier league table and without a trophy last term after having reached the champions league final before losing to real madrid', 'category': 'sports'},
	{'paragraph': 'Liverpool fc will return to premier league action on saturday lunchtime when they travel to leicester city in the top flight as they look to make it four wins in a row in the league', 'category': 'sports'},
	{'paragraph': 'Alisson signed for liverpool fc from as roma this summer and the brazilian goalkeeper has helped the reds to keep three clean sheets in their first three premier league games', 'category': 'sports'},
	{'paragraph': 'But the rankings during that run-in to new york hid some very different undercurrents for murray had struggled with a hip injury since the clay swing and had not played a match since losing his quarter-final at wimbledon and he would pull out of the us open just two days before the tournament began—too late however to promote nederer to the no2 seeding', 'category': 'sports'},
	{'paragraph': 'Then came the oh-so-familiar djokovic-nadal no-quarter-given battle for dominance in the third set there were exhilarating rallies with both chasing to the net both retrieving what looked like winning shots nadal more than once pulled off a reverse smash and had his chance to seal the tie-break but it was djokovic serving at 10-9 who dragged one decisive error from nadal for a two-sets lead', 'category': 'sports'}
]

In [3]:
df = pd.DataFrame(paragraph_dict_list)
df.head()

,paragraph,category
0,Dishplace is located in sunnyvale downtown the...,food
1,Service can be slower during busy hours but ou...,food
2,Portions are huge both french toast and their ...,food
3,We started with apps going the chicken and waf...,food
4,The biscuits and gravy was too salty two peopl...,food


***
데이터전처리
***
LSTM 모델이 입력 데이터를 처리할 수 있도록 수치값으로 변경한다.  
텍스트인 입력값을 수치로 변환하기 위해서 지문에 사용된 모든 단어들을 모아서 중복을 제거한 후 단어 리스트를 만든다.

In [4]:
# 중복을 해서 지문에 사용된 단어를 모으기 위해 빈 set을 선언한다.
results = set()

# 데이터프레임의 paragraph열(Series)을 모두 소문자로 변환한 후 공백을 경계로 나눠서 set에 저장한다.
#print(type(results)) # class 'set'
#print(type(df)) # dataframe
#print(type(df.paragraph)) # Series
#print(type(df.paragraph.str)) #StringMethod
#print(type(df.paragraph.str.lower())) #Series
#print(type(df.paragraph.str.lower().str))# StringMethod
#print(type(df.paragraph.str.lower().str.split(' '))) # Series
df.paragraph.str.lower().str.split(' ').apply(results.update)
print(len(results))
print(results)

536
{'the', 'important', 'busy', 'had', 'modric', 'ice', 'croatia', 'murray', 'form', 'patience', 'delivered', 'french', 'seeded', 'recommended', 'palo', 'began', 'sloane', 'perseyside', 'tea', 'brazilian', 'down', 'england', 'places', 'confidence', 'expensive', 'return', 'five', 'they', 'year', 'dismal', 'halep', 'themselves', 'their', '14', 'spot', 'soon', 'probably', 'two', '2', 'bottom', 'tests', 'certainly', 'business', 'reservations', 'australia', 'chances', 'chicken', 'up', 'making', 'where', 'courts', 'courteous', 'didnt', 'djokovic-nadal', 'brunch', 'years', 'separate', 'alto', 'wanted', 'dog-friendly', 'around', 'what', 'hold', 'breakthroughs', 'dragged', 'crisp', 'victoria', 'after', 'flair', 'wozniacki', 'win', 'rich', 'including', 'burger', 'meal', 'hope', 'smash', 'i', 'too', 'fellow', 'happy', 'got', 'much', 'portions', 'performances', 'angelique', 'food', 'retrieving', 'reverse', '–', 'tournament', 'waiter', 'soft', 'konta', 'slow', 'things', 'she', 'dishplace', 'sat', 

***
set에 저장된 데이터를 아래와 같은 형태의 딕셔너리로 만든다.
***
{ 0 : 'year', 1 : 'tested', 2 : 'gravy', .... , 535 : 'left'}

In [5]:
# enumerate() 함수는 인수로 지정된 객체에 저장된 내용을 (인덱스, 데이터) 형태로 리턴한다. 
#for index, word in enumerate(results):
     #print(index, word)
idx2word = dict(enumerate(results))
print(type(idx2word))
print(idx2word)

<class 'dict'>
{0: 'the', 1: 'important', 2: 'busy', 3: 'had', 4: 'modric', 5: 'ice', 6: 'croatia', 7: 'murray', 8: 'form', 9: 'patience', 10: 'delivered', 11: 'french', 12: 'seeded', 13: 'recommended', 14: 'palo', 15: 'began', 16: 'sloane', 17: 'perseyside', 18: 'tea', 19: 'brazilian', 20: 'down', 21: 'england', 22: 'places', 23: 'confidence', 24: 'expensive', 25: 'return', 26: 'five', 27: 'they', 28: 'year', 29: 'dismal', 30: 'halep', 31: 'themselves', 32: 'their', 33: '14', 34: 'spot', 35: 'soon', 36: 'probably', 37: 'two', 38: '2', 39: 'bottom', 40: 'tests', 41: 'certainly', 42: 'business', 43: 'reservations', 44: 'australia', 45: 'chances', 46: 'chicken', 47: 'up', 48: 'making', 49: 'where', 50: 'courts', 51: 'courteous', 52: 'didnt', 53: 'djokovic-nadal', 54: 'brunch', 55: 'years', 56: 'separate', 57: 'alto', 58: 'wanted', 59: 'dog-friendly', 60: 'around', 61: 'what', 62: 'hold', 63: 'breakthroughs', 64: 'dragged', 65: 'crisp', 66: 'victoria', 67: 'after', 68: 'flair', 69: 'wozni

***
{ 0 : 'year', 1 : 'tested', 2 : 'gravy', .... , 535 : 'left'} 형태의 딕셔너리를  
{ 'year' ; 0, 'tested' : 1,  'gravy' : 2, .... , 'left' : 535 } 형태의 딕셔너리로 변환한다.
***

In [6]:
#word2idx = {}
#for key in idx2word.keys():
     #print(key, idx2word[key])
     #word2idx[idx2word[key].strip()] = key
#for key, value in idx2word.items():
     #print(key, value)
     #word2idx[value.strip()] = key
#word2idx = { value.strip() : key for key, value in idx2word.items() }
word2idx = { value.strip() : index for index, value in enumerate(results) }
print(word2idx)

{'the': 0, 'important': 1, 'busy': 2, 'had': 3, 'modric': 4, 'ice': 5, 'croatia': 6, 'murray': 7, 'form': 8, 'patience': 9, 'delivered': 10, 'french': 11, 'seeded': 12, 'recommended': 13, 'palo': 14, 'began': 15, 'sloane': 16, 'perseyside': 17, 'tea': 18, 'brazilian': 19, 'down': 20, 'england': 21, 'places': 22, 'confidence': 23, 'expensive': 24, 'return': 25, 'five': 26, 'they': 27, 'year': 28, 'dismal': 29, 'halep': 30, 'themselves': 31, 'their': 32, '14': 33, 'spot': 34, 'soon': 35, 'probably': 36, 'two': 37, '2': 38, 'bottom': 39, 'tests': 40, 'certainly': 41, 'business': 42, 'reservations': 43, 'australia': 44, 'chances': 45, 'chicken': 46, 'up': 47, 'making': 48, 'where': 49, 'courts': 50, 'courteous': 51, 'didnt': 52, 'djokovic-nadal': 53, 'brunch': 54, 'years': 55, 'separate': 56, 'alto': 57, 'wanted': 58, 'dog-friendly': 59, 'around': 60, 'what': 61, 'hold': 62, 'breakthroughs': 63, 'dragged': 64, 'crisp': 65, 'victoria': 66, 'after': 67, 'flair': 68, 'wozniacki': 69, 'win': 7

***
word2idx 딕셔너리를 활용해서 모든 지문을 구성하는 단어 목록을 수치로 변환한 파생 변수를 데이터프레임에 추가한다.
***

In [7]:
# Dishplace is located in sunnyvale downtown -> 302 462 529 284 510 294
print(word2idx['dishplace'], word2idx['is'], word2idx['located'], word2idx['in'], word2idx['sunnyvale'], word2idx['downtown'])

97 145 218 374 515 138


In [8]:
#지문을 구성하는 모든 단어를 수치로 변환하는 함수를 만든다.
def encode_paragraph(paragragh):
     #print(type(paragraph), paragragh)
     words = paragragh.split()
     #print(words)
     encoded = []
     for word in words:
          encoded.append([word2idx[word.lower()]])
     return encoded;

In [9]:
# 지문을 수치로 변환된 결과를 데이터 프레임에 파생변수로 추가한다. -> [[302], [462], [529], [284], [510], [294]]
df['enc_paragraph'] = df.paragraph.apply(encode_paragraph)
df

,paragraph,category,enc_paragraph
0,Dishplace is located in sunnyvale downtown the...,food,"[[97], [145], [218], [374], [515], [138], [494..."
1,Service can be slower during busy hours but ou...,food,"[[207], [365], [134], [522], [161], [2], [528]..."
2,Portions are huge both french toast and their ...,food,"[[83], [532], [273], [366], [11], [270], [474]..."
3,We started with apps going the chicken and waf...,food,"[[453], [160], [466], [146], [330], [0], [46],..."
4,The biscuits and gravy was too salty two peopl...,food,"[[0], [390], [474], [500], [287], [78], [502],..."
5,The garlic fries were a great starter (and a h...,food,"[[0], [229], [479], [475], [112], [513], [209]..."
6,Our meal was excellent i had the pasta ai form...,food,"[[103], [74], [287], [465], [77], [3], [0], [5..."
7,What i enjoy most about palo alto is so many r...,food,"[[61], [77], [262], [183], [189], [14], [57], ..."
8,The drinks came out fairly quickly a good two ...,food,"[[0], [341], [152], [155], [434], [364], [112]..."
9,Despite the not so good burger the service was...,food,"[[326], [0], [347], [175], [501], [73], [0], [..."


***
분류 항목(food -> [1, 0], sports -> [0, 1])을 원-핫 인코딩으로 수치로 변환한 파생변수를 데이터프레임에 추가한다.
***

In [10]:
# 분류 항목을 원-핫 인코딩 수치로 변환하는 함수를 만든다.
def encode_category(category):
     if category == 'food':
          return [1, 0]
     else :
          return [0, 1]


In [11]:
# 분류 항목이 수치로 변환된 결과를 데이터프레임에 추가한다.
df['enc_category'] = df.category.apply(encode_category)
df

,paragraph,category,enc_paragraph,enc_category
0,Dishplace is located in sunnyvale downtown the...,food,"[[97], [145], [218], [374], [515], [138], [494...","[1, 0]"
1,Service can be slower during busy hours but ou...,food,"[[207], [365], [134], [522], [161], [2], [528]...","[1, 0]"
2,Portions are huge both french toast and their ...,food,"[[83], [532], [273], [366], [11], [270], [474]...","[1, 0]"
3,We started with apps going the chicken and waf...,food,"[[453], [160], [466], [146], [330], [0], [46],...","[1, 0]"
4,The biscuits and gravy was too salty two peopl...,food,"[[0], [390], [474], [500], [287], [78], [502],...","[1, 0]"
5,The garlic fries were a great starter (and a h...,food,"[[0], [229], [479], [475], [112], [513], [209]...","[1, 0]"
6,Our meal was excellent i had the pasta ai form...,food,"[[103], [74], [287], [465], [77], [3], [0], [5...","[1, 0]"
7,What i enjoy most about palo alto is so many r...,food,"[[61], [77], [262], [183], [189], [14], [57], ...","[1, 0]"
8,The drinks came out fairly quickly a good two ...,food,"[[0], [341], [152], [155], [434], [364], [112]...","[1, 0]"
9,Despite the not so good burger the service was...,food,"[[326], [0], [347], [175], [501], [73], [0], [...","[1, 0]"


***
각 입력값의 실제 길이(단어 갯수)를 알 수 있도록 각 지문별 단어의 갯수를 파생변수로 데이터프레임에 추가한다.
***

In [12]:
# 지문을 구성하는 단어의 갯수를 세는 함수를 만든다.
def word_cnt(paragraph):
     #print(paragraph.split())
     #print('////')
     return len(paragraph.split())

In [13]:
# 지문을 구성하는 단어의 갯수를 데이터프레임에 파생변수로 추가한다.
df['seq_length'] = df.paragraph.apply(word_cnt)
df

,paragraph,category,enc_paragraph,enc_category,seq_length
0,Dishplace is located in sunnyvale downtown the...,food,"[[97], [145], [218], [374], [515], [138], [494...","[1, 0]",53
1,Service can be slower during busy hours but ou...,food,"[[207], [365], [134], [522], [161], [2], [528]...","[1, 0]",19
2,Portions are huge both french toast and their ...,food,"[[83], [532], [273], [366], [11], [270], [474]...","[1, 0]",42
3,We started with apps going the chicken and waf...,food,"[[453], [160], [466], [146], [330], [0], [46],...","[1, 0]",43
4,The biscuits and gravy was too salty two peopl...,food,"[[0], [390], [474], [500], [287], [78], [502],...","[1, 0]",82
5,The garlic fries were a great starter (and a h...,food,"[[0], [229], [479], [475], [112], [513], [209]...","[1, 0]",24
6,Our meal was excellent i had the pasta ai form...,food,"[[103], [74], [287], [465], [77], [3], [0], [5...","[1, 0]",50
7,What i enjoy most about palo alto is so many r...,food,"[[61], [77], [262], [183], [189], [14], [57], ...","[1, 0]",43
8,The drinks came out fairly quickly a good two ...,food,"[[0], [341], [152], [155], [434], [364], [112]...","[1, 0]",49
9,Despite the not so good burger the service was...,food,"[[326], [0], [347], [175], [501], [73], [0], [...","[1, 0]",82


***
RNN은 항상 같은 길이의 시퀀스를 받아야 한다.; 입력되는 값의 길이가 모두 같아야함  
길이가 작은 입력 시퀀스는 패딩을 추가적으로 넣어서 모든 시퀀스 길이를 동일하게 설정한다.  
패딩이 RNN 계산에 영향을 끼치지 않도록 패딩 이전의 입력 시퀀스의 실제 길이를 파라미터로 받아 상태값(출력값) 계산시 패딩을 제외한다.  
***

In [14]:
# 최고로 긴 지문의 단어의 수를 구한 후 모든 지문에 패딩을 집어넣어 최고로 긴 지문과 동일한 길이를 갖도록 만들기 위해 
# 최고로 긴 지문의 단어의 수를 계산한다.
#max_word_cnt = 0
#for row in df.paragraph:
     #print(type(row), row)
     #print('/////')
     #if len(row.split()) > max_word_cnt :
          #max_word_cnt = len(row.split())
     #--- if len
#for seq in df.seq_length:
#     if seq > max_word_cnt:
#          max_word_cnt = seq

max_word_cnt = df.seq_length.max()
print(max_word_cnt)

91


In [15]:
# 모든 지문에 패딩([-1])을 집어넣어 최고로 긴 문장과 동일한 길이를 갖게하는 함수를 만든다.
def sequence_padding (enc_paragraph):
     #print(type(enc_paragraph), enc_paragraph)
     #print('/////')
     #print(len(enc_paragraph))
     for i in range(len(enc_paragraph), max_word_cnt):
          enc_paragraph.append([-1])
     
     #for _ in range(max_word_cnt - len(enc_paragraph)):
     #     enc_paragraph.append([-1])
     #---- for _
     return enc_paragraph
     print(enc_paragraph)
     #print(len(enc_paragraph))
     #print('//////')

In [16]:
# 모든 지문에 패딩이 적용된 결과를 enc_paragraph 컬럼에 적용한다.
#df['encode_paragraph'] = df.enc_paragraph.apply(sequence_padding)
df.enc_paragraph.apply(sequence_padding)
print(len(df.enc_paragraph[1]))
print(df.enc_paragraph[1])
print(len(df.enc_paragraph[12]))
print(df.enc_paragraph[12])

91
[[207], [365], [134], [522], [161], [2], [528], [174], [103], [91], [287], [51], [474], [219], [497], [234], [513], [182], [141], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1]]
91
[[460], [245], [512], [194], [165], [404], [100], [374], [371], [212], [331], [70], [512], [395], [269], [489], [354], [508], [389], [278], [484], [96], [363], [534], [349], [61], [112], [308], [179], [292], [96], [145], [455], [0], [387], [250], [488], [360], [79], [269], [383], [271], [405], [0], [166], [272], [0], [11], [396], [165], [169], [346], [461], [204], [50], [96], [392], [331], [0], [166], [374], [473], [181], [278],

***
모델에서 사용할 입력값, 실제값(출력값), 지문의 실제 길이 파라미터를 만든다.
***

In [17]:
# numpy의 array() 메소드는 인수로 파이썬의 리스트나 튜플 데이터를 받아서 numpy 배열로 변환하는 기능을 하기때문에
# 데이터프레임의 시리즈를 바로 넣어주면 안되고, tolist() 메소드를 실행해서 파이썬의 리스트 형태로 변환한 후 넣어준다.
#print(type(df.enc_paragraph)) #class 'Series'
#print(type(df.enc_paragraph.to_list())) #class 'list'
enc_paragraph = np.array(df.enc_paragraph.tolist()) # 입력값
#print(enc_paragraph) # np.array
#print(enc_paragraph)
enc_category = np.array(df.enc_category.tolist()) # 실제값;레이블
#print(enc_category)
seq_length = np.array(df.seq_length.tolist()) # 지문의 실제 길이
#print(seq_length)


x_train = enc_paragraph
y_train = enc_category
print(x_train.shape, y_train.shape)

(20, 91, 1) (20, 2)


***
문맥 벡터(Contextualized Vector) 생성 단계
***
인덱스를 임베딩으로 변환한다.  
임베딩은 학습과정을 통해 단어 유사도를 포함하게 되고 문맥 벡터를 생성하는데 도움을 준다.  
인간의 언어(자연어)는 수치화돼 있지 않은 데이터이기때문에 머신러닝, 딥러닝 기법을 바로 사용할 수 없다.  
그래서 자연어 처리에서 특징을 추출해서 수치화를 해줘야하는데 이때 사용하는 것이 '언어의 벡터화'이다. 이런 벡터화 과정을 word embedding이라고 한다.  
LSTM에서 임베딩된 시퀀스를 입력해서 최종 상태값을 출력한다. -> 최종 상태값이 문맥 벡터이다.  

***
주제(food, sports) 분류 단계
***
문맥 벡터를 덴즈 레이어에 입력하고 출력값을 노드가 2개인 덴즈 레이어에 입력한 후 노드가 2개인 덴즈 레이어의 출력값을 소프트 맥에 입력시켜  
food, sports에 대한 예측값을 계산한다.



***
지문을 읽고 주제를 분류하는 LSTM 모델을 만든다.
***

<img src="LSTM (1).png">

***
LSTM 모델을 구현한다.
***

In [18]:
tf.set_random_seed(35)

#입력값(enc_paragraph)과 실제값(enc_category)을 저장할 placeholder를 만든다.
# 입력값은 문장을 구성하는 단어들의 인덱스이며, 그 길이는 지문을 구성하는 단어의 최대 갯수(max_word_cnt)이다.
x = tf.placeholder(dtype=tf.float32, shape=(None, max_word_cnt, 1)) #입력값을 기억할 placeholder (20, 91, 1)
y = tf.placeholder(dtype=tf.float32, shape=(None, 2)) #실제값을 기억할 placeholder (20, 2) 20은 자료의 갯수; 후에 증가할지 감소할지 모르므로 placeholder -> none

#임베딩 레이어는 입력값(단어들의 인덱스)을 입력받아 5차원의 벡터 임베딩을 출력한다.
#layers.dense() 메소드는 densely-connect layer 즉, '완전 연결 계층'을 만들어 준다.
embedding = tf.layers.dense(x, 5)

#LSTM 셀은 64차원의 벡터의 생성값을 출력한다.
cell = tf.nn.rnn_cell.LSTMCell(num_units=64)

# LSTM 셀의 출력값과 상태값을 저장한다.
outputs, state = tf.nn.dynamic_rnn(cell=cell, dtype=tf.float32, inputs=embedding, sequence_length=seq_length)

# 주제 분류는 두개의 덴즈 레이어를 사용한다.
# 첫번째 덴즈 레이어는 32개의 노드를 가지고 있고, 두번쨰 덴즈 레이어는 2개의 노드를 가지고 있으며
# 이 2개의 노드가 소프트 맥스의 입력으로 들어간다.
dense_layer = tf.layers.dense(state.h, 32) # c -> 메모리셀 값, h -> 실제 출력값

# logit은 food, sports를 원-핫 인코딩으로 구분하기위해 2차원 벡터로 구성한다. 
logit = tf.layers.dense(dense_layer, 2) # 최종 예측값

# 손실 함수 -> 크로스 엔트로피
loss = tf.reduce_mean( 
     tf.nn.softmax_cross_entropy_with_logits_v2(logits=logit, labels=y)
)

# Adam 옵티마이저
train = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)

Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


***
모델 요약
***

In [19]:
#문맥 벡터 생성
#입력값은 단어들의 인덱스이며 그 길이는 항상 91이다.
print(x) #Tensor("Placeholder:0", shape=(?, 91, 1), dtype=float32)

#임베딩 레이어는 인덱스를 받아 5차원 벡터의 임베딩을 출력한다.
print(embedding) #Tensor("dense/BiasAdd:0", shape=(?, 91, 5), dtype=float32)

# LSTM 셀은 64차원의 상태값을 출력한다.
print(state.h) #Tensor("rnn/while/Exit_4:0", shape=(?, 64), dtype=float32)


#문맥 벡터를 사용해서 지문의 주제 분류하기, 주제 분류에는 2개의 덴즈 레이어를 사용했다.
#첫번째 덴즈 레이어는 32개의 노드를 가진다. 
print(dense_layer) #Tensor("dense_1/BiasAdd:0", shape=(?, 32), dtype=float32)

#두번째 덴즈 레이어는 2개의 노드를 가지고 있으며, 이 2개의 노드는 소프트 맥스의 입력으로 들어간다.
print(logit) #Tensor("dense_2/BiasAdd:0", shape=(?, 2), dtype=float32)

Tensor("Placeholder:0", shape=(?, 91, 1), dtype=float32)
Tensor("dense/BiasAdd:0", shape=(?, 91, 5), dtype=float32)
Tensor("rnn/while/Exit_4:0", shape=(?, 64), dtype=float32)
Tensor("dense_1/BiasAdd:0", shape=(?, 32), dtype=float32)
Tensor("dense_2/BiasAdd:0", shape=(?, 2), dtype=float32)


***
학습시킨다.
***

In [20]:
with tf.Session() as sess:
     sess.run(tf.global_variables_initializer())
     for epoch in range(401):
          _, loss_ = sess.run([train, loss], feed_dict={x: x_train, y: y_train})
          
          
          if epoch % 20 == 0:
               predict = tf.nn.softmax(logits=logit) #최종 예측값
               # 정확도를 계산하는 수식을 만든다.
               correct_predict = tf.equal(tf.argmax(predict, axis=1), tf.argmax(y, axis=1))
               accuracy = tf.reduce_mean(tf.cast(correct_predict, dtype=tf.float32))
               current_accuracy = accuracy.eval(feed_dict={x: x_train, y: y_train}) # 정확도 계산
               print('epoch -> {:3d}, loss(오차) -> {:6.4f}, accuracy(정확도) -> {:6.4f}'.format(epoch, loss_, current_accuracy))

epoch ->   0, loss(오차) -> 0.7384, accuracy(정확도) -> 0.5000
epoch ->  20, loss(오차) -> 0.5219, accuracy(정확도) -> 0.8500
epoch ->  40, loss(오차) -> 0.3175, accuracy(정확도) -> 0.9000
epoch ->  60, loss(오차) -> 0.0900, accuracy(정확도) -> 1.0000
epoch ->  80, loss(오차) -> 0.0183, accuracy(정확도) -> 1.0000
epoch -> 100, loss(오차) -> 0.0057, accuracy(정확도) -> 1.0000
epoch -> 120, loss(오차) -> 0.0026, accuracy(정확도) -> 1.0000
epoch -> 140, loss(오차) -> 0.0015, accuracy(정확도) -> 1.0000
epoch -> 160, loss(오차) -> 0.0009, accuracy(정확도) -> 1.0000
epoch -> 180, loss(오차) -> 0.0009, accuracy(정확도) -> 1.0000
epoch -> 200, loss(오차) -> 0.0004, accuracy(정확도) -> 1.0000
epoch -> 220, loss(오차) -> 0.0003, accuracy(정확도) -> 1.0000
epoch -> 240, loss(오차) -> 0.0002, accuracy(정확도) -> 1.0000
epoch -> 260, loss(오차) -> 0.0002, accuracy(정확도) -> 1.0000
epoch -> 280, loss(오차) -> 0.0001, accuracy(정확도) -> 1.0000
epoch -> 300, loss(오차) -> 0.0001, accuracy(정확도) -> 1.0000
epoch -> 320, loss(오차) -> 0.0001, accuracy(정확도) -> 1.0000
epoch -> 340, 